# Load Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown
plt.style.use('ggplot')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/dacon_anomaly')

In [4]:
def load_data():
    train = pd.read_csv('./data/train.csv')
    valid = pd.read_csv('./data/val.csv')
    test = pd.read_csv('./data/test.csv')

    return train, valid, test

train, valid, test = load_data()

In [5]:
test

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,AAAA0x1,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.783274,-0.994983
1,AAAA0x2,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.269825,-0.994983
2,AAAA0x5,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.670579,-0.994960
3,AAAA0x7,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,-0.237686,-0.994937
4,AAAA0xc,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,-0.167819,-0.994866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142498,0x4587f,0.219529,0.881246,-0.635891,0.960928,-0.152971,-1.014307,0.427126,0.121340,-0.285670,...,0.099936,0.337120,0.251791,0.057688,-1.508368,0.144023,0.181205,0.215243,0.028645,1.034904
142499,0x45880,-1.775135,-0.004235,1.189786,0.331096,1.196063,5.519980,-1.518185,2.080825,1.159498,...,0.103302,0.654850,-0.348929,0.745323,0.704545,-0.127579,0.454379,0.130308,0.810312,1.034916
142500,0x45884,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0.038986,1.034963
142501,0x45885,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0.641096,1.034975


In [6]:
def preprocess(train, valid, test):
    train.drop(['ID'], inplace = True, axis = 1)
    valid.drop(['ID'], inplace = True, axis = 1)
    test.drop(['ID'], inplace = True, axis = 1)

    X_train = train.values
    X_valid = valid.drop(['Class'], axis = 1).values
    y_valid = valid['Class'].values
    X_test = test.values

    return X_train, X_valid, y_valid, X_test

X_train, X_valid, y_valid, X_test = preprocess(train, valid, test)

# AutoEncoder with sub-sampling
- 학습과정에서 모델의 크기가 커지거나 epoch수가 늘어나면 f1 score가 감소함
    - 모델이 이상치까지 학습하는 것으로 볼 수 있음

- Isolation Forest 논문에서 제시한 sub-sampling 기법을 autoencoder에 적용
    - sub-sampling을 통해서 이상치 탐지에 있어 `masking problem`과 `swamping preblem`을 완화할 수 있다.

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import argparse
from tqdm import tqdm
import random
from itertools import product
from sklearn.metrics import f1_score, confusion_matrix

In [8]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = True

set_seed(123)

In [66]:
class AutoEncoder(nn.Module):

    def __init__(self, input_size, encoder_hidden_size, bottle_neck_size, decoder_hidden_size, dropout_p):
        self.input_size = input_size
        self.encoder_hidden_size = encoder_hidden_size
        self.bottle_neck_size = bottle_neck_size
        self.decoder_hidden_size = decoder_hidden_size
        self.dropout_p = dropout_p

        super(AutoEncoder, self).__init__()


        # ===== Encoder ===== #
        self.encoder = nn.ModuleList()
        # input layer
        self.encoder.append(nn.Sequential(
            nn.Linear(input_size, encoder_hidden_size[0]),
            nn.BatchNorm1d(encoder_hidden_size[0]),
            nn.ReLU(),
            nn.Dropout(dropout_p)
        ))
        # encoder hidden layers
        for idx in range(len(encoder_hidden_size) - 1):
            self.encoder.append(nn.Sequential(
                nn.Linear(encoder_hidden_size[idx], encoder_hidden_size[idx + 1]),
                nn.BatchNorm1d(encoder_hidden_size[idx + 1]),
                nn.ReLU(),
                nn.Dropout(dropout_p)
            ))
        # bottle neck layer
        self.encoder.append(nn.Sequential(
            nn.Linear(encoder_hidden_size[-1], bottle_neck_size),
            nn.BatchNorm1d(bottle_neck_size),
            nn.ReLU(),
            nn.Dropout(dropout_p)
        ))


        # ===== Decoder ===== #
        self.decoder = nn.ModuleList()
        # bottle neck
        self.decoder.append(nn.Sequential(
            nn.Linear(bottle_neck_size, decoder_hidden_size[0]),
            nn.BatchNorm1d(decoder_hidden_size[0]),
            nn.ReLU(),
            nn.Dropout(dropout_p)
        ))
        # decoder hidden layers
        for idx in range(len(decoder_hidden_size) - 1):
            self.decoder.append(nn.Sequential(
                nn.Linear(decoder_hidden_size[idx], decoder_hidden_size[idx + 1]),
                nn.BatchNorm1d(decoder_hidden_size[idx + 1]),
                nn.ReLU(),
                nn.Dropout(dropout_p)
            ))
        # output_layer
        self.decoder.append(nn.Linear(decoder_hidden_size[-1], input_size))


    def forward(self, x):
        # |x| = (batch_size, input_size)
        for layer in self.encoder:
            x = layer(x)
        
        latent = x
        # |latent| = (batch_size, bottle_neck_size)

        for layer in self.decoder:
            x = layer(x)
        # |x| = (batch_size, input_size)

        return x


class AutoEncoderWithSubSampling(nn.Module):

    def __init__(self, n_estimators, input_size, encoder_hidden_size, bottle_neck_size, decoder_hidden_size, dropout_p):
        self.n_estimators = n_estimators
        self.input_size = input_size
        self.encoder_hidden_size = encoder_hidden_size
        self.bottle_neck_size = bottle_neck_size
        self.decoder_hidden_size = decoder_hidden_size
        self.dropout_p = dropout_p

        super(AutoEncoderWithSubSampling, self).__init__()

        self.estimators = [
            AutoEncoder(
                input_size = input_size,
                encoder_hidden_size = encoder_hidden_size,
                bottle_neck_size = bottle_neck_size,
                decoder_hidden_size = decoder_hidden_size,
                dropout_p = dropout_p
            ) for _ in range(n_estimators)
        ]
    
    def forward(self):
        pass

    def train(self, train_loaders, valid_loader, y_valid, n_epochs, device):
        for idx in range(len(train_loaders)):
            display(Markdown('# Estimator {}/{}'.format((idx + 1), len(train_loaders))))

            sub_model = self.estimators[idx]
            sub_model.to(device)
            sub_loader = train_loaders[idx]

            optimizer = optim.Adam(sub_model.parameters())
            crit = nn.L1Loss()
            cos = nn.CosineSimilarity(dim = 1)


            best_f1 = -np.inf
            best_model = None
            for epoch in range(n_epochs):
                train_losses = []
                valid_losses = []
                preds = []
                # === train === #
                for batch in sub_loader:
                    batch = batch.float().to(device)

                    # initialize optimizer
                    optimizer.zero_grad()

                    # feed foward
                    x_hat = sub_model(batch)

                    # loss
                    loss = crit(batch, x_hat)

                    # backpropagation
                    loss.backward()

                    # gradient descent
                    optimizer.step()

                    train_losses.append(float(loss))

                # === valid === #                
                for batch in valid_loader:
                    batch = batch.float().to(device)

                    sub_model.eval()
                    with torch.no_grad():
                        # feed foward
                        x_hat = sub_model(batch)

                        # loss
                        loss = crit(batch, x_hat) 
                        valid_losses.append(float(loss))

                        # cosine similarity
                        sim = cos(batch, x_hat).detach().cpu().numpy()

                        pred = (sim < 0.95)
                        preds.extend(pred)

                train_loss = np.mean(train_losses)
                valid_loss = np.mean(valid_losses)
                f1 = f1_score(y_valid, preds, average = 'macro')

                if f1 > best_f1:
                    best_model = sub_model.state_dict()
                    best_f1 = f1

                if epoch % 20 == 0:
                    print(f'Epoch {epoch + 1} - Loss {np.round(train_loss, 5)}, Val_Loss {np.round(valid_loss, 5)}, f1_score {np.round(f1, 5)}')

    def evaluate(self, X, batch_size, device):
        ensemble = []

        cos = nn.CosineSimilarity(dim = 1)
        
        for estimator in self.estimators:
            estimator.to(device)

            sims = []

            estimator.eval()
            with torch.no_grad():
                for idx in range(0, len(X), batch_size):
                    batch = X[idx: idx + batch_size]
                    batch = torch.FloatTensor(batch).to(device)

                    x_hat = estimator(batch)

                    sim = cos(batch, x_hat).detach().cpu().numpy()

                    sims.extend(sim)
            
            ensemble.append(sims)
        
        ave_sims = np.mean(ensemble, axis = 0)

        pred = (ave_sims < 0.95)

        return pred

In [67]:
class AE_Dataset(Dataset):

    def __init__(self, x):
        self.x = x
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx]


def get_loader(train, valid, batch_size, shuffle = True):
    train_loader = DataLoader(
        AE_Dataset(train),
        batch_size = batch_size,
        shuffle = shuffle
    )
    valid_loader = DataLoader(
        AE_Dataset(valid),
        batch_size = batch_size,
        shuffle = False
    )    

    return train_loader, valid_loader

def sub_sampler(train, valid, batch_size, sampling_size, shuffle  = True):
    train_loaders = []

    # ===== sub-sampling ===== #
    # shuffle data
    if shuffle:
        indices = np.random.permutation(len(train))
        train = train[indices]
    
    # split data
    for idx in range(0, len(train), sampling_size):
        sampled_data = train[idx : idx + sampling_size]

        sampled_loader = DataLoader(
            AE_Dataset(sampled_data),
            batch_size = batch_size,
            shuffle = shuffle
        )
        train_loaders.append(sampled_loader)

    # ===== valid loader ===== #
    valid_loader = DataLoader(
        AE_Dataset(valid),
        batch_size = batch_size,
        shuffle = False
    )

    return train_loaders, valid_loader

In [72]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loaders, valid_loader = sub_sampler(
    X_train, X_valid,
    batch_size = 512,
    sampling_size = 8192,
    shuffle = True
)

model = AutoEncoderWithSubSampling(
    n_estimators = len(train_loaders),
    input_size = 30,
    encoder_hidden_size = [60],
    bottle_neck_size = 120,
    decoder_hidden_size = [60],
    dropout_p = .2
)

model.train(
    train_loaders,
    valid_loader,
    y_valid,
    n_epochs = 200,
    device = DEVICE
)

# Estimator 1/14

Epoch 1 - Loss 0.69355, Val_Loss 0.63106, f1_score 0.00105
Epoch 21 - Loss 0.08638, Val_Loss 0.08794, f1_score 0.52064
Epoch 41 - Loss 0.05217, Val_Loss 0.05498, f1_score 0.80837
Epoch 61 - Loss 0.04467, Val_Loss 0.04637, f1_score 0.89671
Epoch 81 - Loss 0.03646, Val_Loss 0.03855, f1_score 0.90974
Epoch 101 - Loss 0.03799, Val_Loss 0.04006, f1_score 0.91658
Epoch 121 - Loss 0.03713, Val_Loss 0.03504, f1_score 0.91658
Epoch 141 - Loss 0.03308, Val_Loss 0.03202, f1_score 0.91658
Epoch 161 - Loss 0.0376, Val_Loss 0.04068, f1_score 0.91658
Epoch 181 - Loss 0.03571, Val_Loss 0.04109, f1_score 0.91658


# Estimator 2/14

Epoch 1 - Loss 0.6851, Val_Loss 0.62352, f1_score 0.00105
Epoch 21 - Loss 0.0865, Val_Loss 0.08306, f1_score 0.53609
Epoch 41 - Loss 0.05322, Val_Loss 0.05595, f1_score 0.63035
Epoch 61 - Loss 0.04345, Val_Loss 0.03763, f1_score 0.83763
Epoch 81 - Loss 0.03491, Val_Loss 0.03474, f1_score 0.88448
Epoch 101 - Loss 0.03339, Val_Loss 0.03819, f1_score 0.88697
Epoch 121 - Loss 0.0367, Val_Loss 0.04398, f1_score 0.69025
Epoch 141 - Loss 0.03349, Val_Loss 0.03298, f1_score 0.53003
Epoch 161 - Loss 0.03351, Val_Loss 0.03347, f1_score 0.53003
Epoch 181 - Loss 0.0348, Val_Loss 0.03679, f1_score 0.53099


# Estimator 3/14

...

Epoch 1 - Loss 0.683, Val_Loss 0.62544, f1_score 0.00105
Epoch 21 - Loss 0.08872, Val_Loss 0.08586, f1_score 0.52185
Epoch 41 - Loss 0.05411, Val_Loss 0.05636, f1_score 0.79381
Epoch 61 - Loss 0.04866, Val_Loss 0.04171, f1_score 0.87083
Epoch 81 - Loss 0.04823, Val_Loss 0.05116, f1_score 0.86829
Epoch 101 - Loss 0.0365, Val_Loss 0.03258, f1_score 0.69025
Epoch 121 - Loss 0.04227, Val_Loss 0.04919, f1_score 0.49974
Epoch 141 - Loss 0.03865, Val_Loss 0.03538, f1_score 0.49974
Epoch 161 - Loss 0.0342, Val_Loss 0.03922, f1_score 0.49974
Epoch 181 - Loss 0.03288, Val_Loss 0.03369, f1_score 0.49974


# Estimator 13/14

Epoch 1 - Loss 0.69262, Val_Loss 0.63386, f1_score 0.00105
Epoch 21 - Loss 0.08491, Val_Loss 0.08875, f1_score 0.52732
Epoch 41 - Loss 0.056, Val_Loss 0.05514, f1_score 0.55507
Epoch 61 - Loss 0.04626, Val_Loss 0.04991, f1_score 0.56571
Epoch 81 - Loss 0.03957, Val_Loss 0.039, f1_score 0.72068
Epoch 101 - Loss 0.04155, Val_Loss 0.04087, f1_score 0.90974
Epoch 121 - Loss 0.03417, Val_Loss 0.03591, f1_score 0.90974
Epoch 141 - Loss 0.03098, Val_Loss 0.03075, f1_score 0.91658
Epoch 161 - Loss 0.03379, Val_Loss 0.03656, f1_score 0.90974
Epoch 181 - Loss 0.03476, Val_Loss 0.0313, f1_score 0.91658


# Estimator 14/14

Epoch 1 - Loss 0.68826, Val_Loss 0.63477, f1_score 0.00105
Epoch 21 - Loss 0.091, Val_Loss 0.09392, f1_score 0.52189
Epoch 41 - Loss 0.05758, Val_Loss 0.05636, f1_score 0.60475
Epoch 61 - Loss 0.0419, Val_Loss 0.04449, f1_score 0.89671
Epoch 81 - Loss 0.04453, Val_Loss 0.04238, f1_score 0.90974
Epoch 101 - Loss 0.0383, Val_Loss 0.03828, f1_score 0.90974
Epoch 121 - Loss 0.03367, Val_Loss 0.03548, f1_score 0.91658
Epoch 141 - Loss 0.03459, Val_Loss 0.03479, f1_score 0.91658
Epoch 161 - Loss 0.03378, Val_Loss 0.03393, f1_score 0.91658
Epoch 181 - Loss 0.03306, Val_Loss 0.03145, f1_score 0.91658


In [73]:
val_pred = model.evaluate(
    X_valid,
    batch_size = 512,
    device = DEVICE
)

In [74]:
f1_score(y_valid, val_pred)

0.8333333333333334

In [75]:
pred = model.evaluate(
    X_test,
    batch_size = 512,
    device = DEVICE
)

In [77]:
submission = pd.read_csv('./data/sample_submission.csv')

In [81]:
submission['Class'] = pred.astype('int')

In [82]:
submission

,ID,Class
0,AAAA0x1,0
1,AAAA0x2,0
2,AAAA0x5,0
3,AAAA0x7,0
4,AAAA0xc,0
...,...,...
142498,0x4587f,0
142499,0x45880,0
142500,0x45884,0
142501,0x45885,0


In [83]:
submission.to_csv('./AE_SubSampling.csv', index = False)